In [1]:
!which python

/home/phillip/.local/share/virtualenvs/actix-telepathy-QCL14L7Z/bin/python


In [2]:
%matplotlib inline
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
conn = sqlite3.connect("results/decentfl.1.db")

exp = pd.read_sql("select * from experiments", conn)
eva = pd.read_sql("select * from evaluations", conn)

In [4]:
exp

,id,cluster_size,lr,batch_size,test_every,update_every,group_size,history_length,dropout,adversarial,krum
0,1,101,0.01,8,1,1,3,1,0.0,0,0
1,2,101,0.01,8,1,1,3,1,0.0,0,0
2,3,101,0.01,8,1,1,3,1,0.0,0,0
3,4,101,0.01,8,1,1,5,1,0.0,0,0
4,5,101,0.01,8,1,1,5,1,0.0,0,0
5,6,101,0.01,8,1,1,5,1,0.0,0,0
6,7,101,0.01,8,1,1,9,1,0.0,0,0
7,8,101,0.01,8,1,1,9,1,0.0,0,0
8,9,101,0.01,8,1,1,9,1,0.0,0,0
9,10,101,0.01,8,1,3,3,1,0.0,0,0


In [18]:
with open("examples/decentfl/migrations/01_add_seed_up.sql", "r") as f:
    sql = f.read()
sql

'ALTER TABLE experiments ADD seed INTEGER;'

In [19]:
conn.execute(sql)

In [ ]:
for r in range(1, 28)